# Imputation of Missing Values with XGBoost

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np

# Load your dataset (replace 'your_dataset.csv' with the actual file path)
data_APT = random_sample

# Columns with missing values
columns_with_missing = ["IR", "Population", "LC_index", "SDT_index", "Crime_Rates"]

# Split into training and imputation sets based on missing values in the target columns
train_set = data_APT.dropna(subset=columns_with_missing)
impute_set = data_APT[data_APT[columns_with_missing].isnull().any(axis=1)]

# Separate features and target variables
X_train = train_set.drop(columns=columns_with_missing)
X_impute = impute_set.drop(columns=columns_with_missing)

# Initialize an empty dictionary to store models
models = {}

# Set parameters for XGBoost model (you can adjust these based on your requirements)
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100  # Use 'n_estimators' instead of 'num_boost_round'
}

# Train separate XGBoost models for each target column
for column in columns_with_missing:
    y_train = train_set[column]
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    models[column] = model

# Predict missing values in the imputation set for each target column
imputed_values = pd.DataFrame()
for column in columns_with_missing:
    model = models[column]
    imputed_values[column] = model.predict(X_impute)

# Replace missing values in the original dataset with the predicted values
data_APT.loc[data_APT[columns_with_missing].isnull().any(axis=1), columns_with_missing] = imputed_values.values

# The missing values have been imputed using XGBoost
nans = data_APT.isnull().sum()
print(nans)
data_APT.info()
data_APT